In [5]:
#import statements
import csv, random
from copy import deepcopy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sympy import *

In [6]:
def cleanData(data):
    #drop bad cols
    new = data.drop(['x2.1', 'x3)'], axis=1)
    #rename cols
    dataframe = new.rename(columns={"f(x1": "f(x)"})
    
    #filter f(x) col for values q1-q3
    fXfilter = dataframe['f(x)'].between(dataframe['f(x)'].quantile(.25), dataframe['f(x)'].quantile(.75))
    # x1filter = dataframe['x1'].between(dataframe['x1'].quantile(.25), dataframe['x1'].quantile(.75))
    # x2filter = dataframe['x2'].between(dataframe['x2'].quantile(.25), dataframe['x2'].quantile(.75))
    # x3filter = dataframe['x3'].between(dataframe['x3'].quantile(.25), dataframe['x3'].quantile(.75))
    #mess around with IQR percentages
   
    for i in range(25000):
        if fXfilter[i] == False:
        # or x1filter[i] == False or x2filter[i] == False or x3filter[i] == False:
            dataframe = dataframe.drop(i)

    dataframe.set_axis(range(len(dataframe)), inplace=True)
    return dataframe
   
def normData(data):
    data_norm = (data-data.min())/(data.max()-data.min())
    print("Normalized data:", data_norm)
    return data_norm

def deNorm(data, xval, col):
    # print("XPRIME = ", xval)
    # print("COL = ", col)
    # print("MAX = ", data[col].max())
    # print("MIN = ", data[col].min())
    de_norm_x = xval*(data[col].max()-data[col].min())+data[col].min()
    #print("NEW X VAL = ", de_norm_x)
    return de_norm_x
        
    

In [7]:
COUNT = [10]
        
class Tree:
    def __init__(self):
        self.root = Node()
        self.operators = ['+', '-', '*', '/']
        self.operands = self.makeOperands()
        self.root.value = self.randomOp()
        self.array = []
        # LOOK AT THIS LATER IF RESULTS ARENT GOOD
        self.fitness = 0
        self.weights = self.makeWeights()
        self._initialTree(self.root)
        
        
    def makeWeights(self):
        #make array of weights for each tree in population
        weightArr = []
        for i in range(100):
            if i <= 20:
                weightArr.append(20)
            elif i > 20 & i<=40:
                weightArr.append(15)
            elif i > 40 & i<=60:
                weightArr.append(5)
            elif i > 60 & i<=80:
                weightArr.append(2)
            elif i > 80 & i<=100:
                weightArr.append(1)
        return weightArr
    
    def makeOperands(self):
        arr = ['x1','x2','x3']
        #arr = ['x3', 'x3']
        for i in range(2):
            arr.append(random.uniform(-5.0,5.0))
        #print(arr)  
        return arr
        
    def randomOp(self):
        #picks a random value to assign to a node
        options = []
        for i in range(len(self.operands)):
            options.append(self.operands[i])
        for i in range(len(self.operators)):
            options.append(self.operators[i])
        length = len(options)
        value = random.randint(0,length-1)
        return options[value]

    def _initialTree(self, node):
        #initalize node values in tree
        
        if (node.value in self.operators):
            leftNode = Node()
            leftNode.value = self.randomOp()
            node.left = leftNode
            leftNode.parent = node
            
            rightNode = Node()
            rightNode.value = self.randomOp()
            node.right = rightNode
            rightNode.parent = node
            
            self._initialTree(leftNode)
            self._initialTree(rightNode)

    def initialPop(self):
        #calls initial tree to create initial population
        forrest = []
        for i in range(10):
            newTree = Tree()
            forrest.append(newTree)

    def printIO(self, node):
        #prints in order traversal of the tree into string
        sVal = ""
        if node:
            sVal = '(' + self.printIO(node.left)

            sVal = sVal + str(node.value)

            sVal = sVal + self.printIO(node.right) + ')'
        return sVal
    
    def treeToArray(self, root):
        #adds elements of tree to array
        if root is None:
            return
        if root:
            self.treeToArray(root.left)

            self.array.append(root)

            self.treeToArray(root.right)
    
    def randomNode(self):
        #picks random node from the tree
        length = len(self.array)
        if length == 1:
            return self.array[0]
        value = random.randint(0,length-1)
        return self.array[value]

    
    def crossover(self, tree2):
        #takes two trees and swaps parent tree 1 with parent tree 2
        
        #makes copy of tree to crossover
        dup1 = deepcopy(self)
        dup1.treeToArray(dup1.root)
        dup2 = deepcopy(tree2)
        dup2.treeToArray(dup2.root)

        #find random node from tree
        random1 = dup1.randomNode()
        random2 = dup2.randomNode()

        #check parents of random nodes and change pointers to right or left
        if random1.parent == None:
            dup1.root = random2
        elif random1.parent.left == random1:
            random1.parent.left = random2
        else:
            random1.parent.right = random2
        
        if random2.parent == None:
            dup2.root = random1
        elif random2.parent.left == random2:
            random2.parent.left = random1
        else:
            random2.parent.right = random1

        tempParent = random1.parent
        random1.parent = random2.parent
        random2.parent = tempParent

        return dup1, dup2
    
    def mutate(self):
        duplicate = deepcopy(self)
        duplicate.treeToArray(duplicate.root)
        
        random1 = duplicate.randomNode()
        
        if random1.value in self.operators:
            length = len(self.operators)
            value = random.randint(0,length-1)
            random1.value = self.operators[value]
        else:
            length = len(self.operands)
            value = random.randint(0,length-1)
            random1.value = self.operands[value]
        
        self._initialTree(random1)
        
        return duplicate
        
    
    def print2DUtil(self, root, space):
        # print tree output 
        
        # Base case
        if (root == None):
            return
        # Increase distance between levels
        space += COUNT[0]
    
        # Process right child first
        self.print2DUtil(root.right, space)
    
        # Print current node after space
        print()
        for i in range(COUNT[0], space):
            print(end=" ")
        print(root.value)
    
        # Process left child
        self.print2DUtil(root.left, space)

    def print2D(self, root):
        # space=[0]
        # Pass initial space count as 0
        self.print2DUtil(root, 0) 
    
    def simplify(self, xVals, node):
        # evaluates tree with given x value
        if node:
            #check for x value
            if node.value == "x1":
                return xVals[0]
            if node.value == "x2":
                return xVals[1]
            if node.value == "x3":
                return xVals[2]
            # check for operand
            if node.value not in self.operators:
                return float(node.value)
            
            # First recur on left child
            valLeft = self.simplify(xVals, node.left)
            # the recur on right child
            valRight = self.simplify(xVals, node.right)
            
            # applying operators
            if node.value == "+":
                return valLeft + valRight
            if node.value == "-":
                return valLeft - valRight
            if node.value == "*":
                return valLeft * valRight
            if node.value == "/":
                if float(valRight) == 0:
                    return 1
                else:
                    return float(valLeft) / float(valRight)
    
    def getFitness(self, data):
        total = 0
        PRESSURE_WEIGHT = 0.001
        #calculates fitness of each tree
        for i in range(len(data)):
            arr = data.iloc[i]
            #print("array", arr)
            #arr['x1'] = float(arr['x1'])
            #arr['x2'] = float(arr['x2'])
            #arr['x3'] = float(arr['x3'])
            #arr['f(x)'] = float(arr['f(x)'])
       
            
            val = self.simplify(arr[0:3], self.root)
            
            output = (val - arr[3])**2
            pressured = output + PRESSURE_WEIGHT*len(self.array)
            total = total + pressured
        # divide by len of training
        self.fitness = total / len(data)
        
    def getFinalFitness(self, reg_data, norm_data):
        total = 0
        PRESSURE_WEIGHT = 0.001
        #calculates fitness of each tree
        for i in range(len(norm_data)):
            norm_arr = norm_data.iloc[i]
            reg_arr = reg_data.iloc[i]
            
            denorm_arr = []
            denorm1 = deNorm(reg_data, norm_arr[0], 'x1')
            denorm_arr.append(denorm1)
            denorm2 = deNorm(reg_data, norm_arr[1], 'x2')
            denorm_arr.append(denorm2)
            denorm3 = deNorm(reg_data, norm_arr[2], 'x3')
            denorm_arr.append(denorm3)
            
            # print("DENORM ARRAY", denorm_arr)
            
            val = self.simplify(denorm_arr[0:3], self.root)
            
            # print("NORM TREE VALUE", val)
            # print("FX GOAL VALUE", reg_arr[3])
            
            output = (val - reg_arr[3])**2
            pressured = output + PRESSURE_WEIGHT*len(self.array)
            total = total + pressured
        # divide by len of training
        self.fitness = total / len(reg_data)
    
    #def finalEval(self, xVals, node, data):
#         # evaluates tree with given x value
#         if node:
#             #check for x value
#             if node.value == "x1":
#                 return deNorm(data, xVals[0], 'x1')
#             if node.value == "x2":
#                 return deNorm(data, xVals[1], 'x2')
#             if node.value == "x3":
#                 return deNorm(data, xVals[2], 'x3')
#             # check for operand
#             if node.value not in self.operators:
#                 return float(node.value)

#             # First recur on left child
#             valLeft = self.finalEval(xVals, node.left, data)
#             # the recur on right child
#             valRight = self.finalEval(xVals, node.right, data)

#             # applying operators
#             if node.value == "+":
#                 return valLeft + valRight
#             if node.value == "-":
#                 return valLeft - valRight
#             if node.value == "*":
#                 return valLeft * valRight
#             if node.value == "/":
#                 if float(valRight) == 0:
#                     return 1
#                 else:
#                     return float(valLeft) / float(valRight)
        

class Node:
    #initialize
    def __init__(self):
        self.parent = None
        self.left = None
        self.right = None
        self.value = None 
        
def experiment(data):
    print("TESTING EVAL")
    # build tree population of 30
    reserve = []
    newGen = []
    for i in range(100):
        newGen.append(Tree())
    
    #producing 100 generations
    NUM_GENS = 0
    while NUM_GENS < 100:
        if NUM_GENS > 20:
            if reserve[NUM_GENS - 1] == reserve[NUM_GENS - 11]:
                # if fitness values have stabilized, regenerate initial population
                newGen = []
                for i in range(100):
                    newGen.append(Tree())
        #loop through each tree
        for tree in range(len(newGen)):
            newGen[tree].getFitness(data)
            #define self.fitness in tree class
        print("FITNESS VALS COMPLETE")

        newGen.sort(key=lambda x: x.fitness)
        print("GEN NUMBER", NUM_GENS)
        print("BEST INDIV FROM NEW GEN", newGen[0].fitness)
        string = newGen[0].printIO(newGen[0].root)
        print("STRING OUTPUT: ", string)
        reserve.append(newGen[0])
        print()
        print()

        # crossover 80
        oldGen = newGen
        newGen = []
        for crossTree in range(40):
            children = []
            childWeights = random.choices(oldGen, weights = oldGen[0].weights, k = 2)

            #value = random.randint(0, len(oldGen)-1)
            
            # generates 10 trees from a crossover of 2 trees
            for brood in range(2):
                cross1, cross2 = childWeights[0].crossover(childWeights[1])
                children.append(cross1)
                children.append(cross2)
                
            # evaluates the fitness of the 10 children generated
            for child in children:
                child.getFitness(data)
            
            children.sort(key=lambda x: x.fitness)
            
            #newGen.append(crossed)
            newGen.append(children[0])
            newGen.append(children[1])
        #print("CROSSOVER COMPLETE")
        #print("NEW GEN AFTER CROSSOVER", newGen[0:10])
        
        # mutate 15
        for mute in range(15):
            # randomize tree chosen for mutate -> fitter individuals have higher chance
            mutated = oldGen[mute].mutate()
            newGen.append(mutated)
        #print("MUTATE COMPLETE")
        #print("NEW GEN AFTER MUTATE", newGen[0:10])

        # reproduce 5
        for repro in range(5):
            # picked with probability proportionate to fitness
            newGen.append(oldGen[repro])
        #print("REPRO COMPLETE")
        
        print()
        print()
        NUM_GENS = NUM_GENS + 1
        #while loop end
    
    reserve.append(newGen[0])
    return reserve

In [12]:
def main():
    # with open('dataset2.csv', newline='') as csvfile:
    #     data = list(csv.reader(csvfile))
    # data = data[1:]
    # train = data[0:20000]
    # print("training:", train[0:10])
    # test = data[20000:]
    
    #dataframe for plots
    df = pd.read_csv("dataset2.csv")
    cleanDF = cleanData(df)
    print(len(cleanDF))
    #row = cleanDF.iloc[0]
    regTrain = cleanDF[0:10000]
    regTest = cleanDF[10000:]
    
    normalized = normData(cleanDF)
    
    miniTrain = normalized[0:10000]
    miniTest = normalized[10000:]
    #train = cleanDF[0:6500]
    #test = cleanDF[6500:]
    
    
#     result = experiment(regTrain)
    
#     result.sort(key=lambda x: x.fitness)
#     print("BEST FITNESS ON MINI TRAINING: ", result[0].fitness)
#     print("IO TRAVERSAL")
#     string = result[0].printIO(result[0].root)
#     print("STRING OUTPUT: ", string)
    
    
    # print("IO TRAVERSAL NORMALIZED")
    # result[0].getFinalFitness(regTrain, miniTrain)
    # print("UPDATED FITNESS ON MINI TRAINING: ", result[0].fitness)
    # string = result[0].printIO(result[0].root)
    # print("STRING OUTPUT: ", string) 
    # print("2D PRINT")
    # result.print2D(result.root)
    
#     print("TESTING RESERVE ON MINI TEST")
#     for i in range(len(result)):
#         result[i].getFitness(regTest)
        
#     result.sort(key=lambda x: x.fitness)
#     print("BEST FITNESS ON TEST: ", result[0].fitness)
#     print("IO TRAVERSAL")
#     string = result[0].printIO(result[0].root)
#     print("STRING OUTPUT: ", string) 
        
    
    #simplify expression
    print("x1 max:, ", cleanDF['x1'].max())
    print("x1 min:, ", cleanDF['x1'].min())
    print("x2 max:, ", cleanDF['x2'].max())
    print("x2 min:, ", cleanDF['x2'].min())
    print("x3 max:, ", cleanDF['x3'].max())
    print("x3 min:, ", cleanDF['x3'].min())
    

    x = symbols('x')
    expression = (((0.3338541282361531)*((x*99992191+6856)*((0.2903189079960944)+(x*988184+10910))))/(x*9.74+0.26))
    output = simplify(expression)
    print("After Simplification : {}".format(output)) 
    
#     # Creating vectors X and Y
#     x = np.linspace(-6, 6, 100)
#     y =  2*(5 - 3*x)/(x + 5)
#     fig = plt.figure(figsize = (10, 5))
#     # Create the plot
#     plt.plot(x, y, color='black')
    
#     plt.scatter(df['x'], df['f(x)'], color='red')
    
#     #adding labels
#     plt.title('Tree Expression and Dataset')
#     plt.xlabel('x value')
#     plt.ylabel('f(x) value')

#     # Show the plot
#     plt.show()
    
    
if __name__ == "__main__":
    main()

12501
Normalized data:              x1        x2        x3      f(x)
0      0.961014  0.225101  0.301848  0.638940
1      0.753540  0.012186  0.077002  0.241830
2      0.201216  0.536328  0.471253  0.073274
3      0.837155  0.831897  0.656057  0.452512
4      0.385930  0.382670  0.671458  0.025123
...         ...       ...       ...       ...
12496  0.618241  0.709893  0.808008  0.142667
12497  0.648996  0.248405  0.672485  0.034722
12498  0.496283  0.672131  0.315195  0.943730
12499  0.468463  0.086749  0.329569  0.032016
12500  0.661637  0.923045  0.525667  0.635059

[12501 rows x 4 columns]
x1 max:,  9999094
x1 min:,  10910
x2 max:,  9999047
x2 min:,  6856
x3 max:,  10.0
x3 min:,  0.26
After Simplification : (32988354523906.4*x**2 + 366477960680.282*x + 24972606.0958526)/(9.74*x + 0.26)
